In [ ]:
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
from random import shuffle


In [ ]:
from tensorflow import keras
# from keras.utils import np_utils
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPool2D, Input, UpSampling2D, Flatten, BatchNormalization, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [ ]:
# We shall not build a new model from scratch, rather we will use transfer learning:

img_size = 224
classes_num = 7
from tensorflow.keras.applications.resnet50 import ResNet50
import tensorflow.keras.applications

resnet_model = ResNet50(input_shape=(img_size, img_size, 3), include_top=False, weights='imagenet')


In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   validation_split = 0.2,
                                  
        rotation_range=5,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        #zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale = 1./255,
                                  validation_split = 0.2)

test_datagen  = ImageDataGenerator(rescale = 1./255)

In [ ]:
train_dataset  = train_datagen.flow_from_directory(directory = '../input/fer2013/train',
                                                   target_size = (48,48),
                                                   class_mode = 'categorical',
                                                   subset = 'training',
                                                   batch_size = 64)

In [ ]:
valid_dataset = valid_datagen.flow_from_directory(directory = '../input/fer2013/train',
                                                  target_size = (48,48),
                                                  class_mode = 'categorical',
                                                  subset = 'validation',
                                                  batch_size = 64)

In [ ]:
test_dataset = test_datagen.flow_from_directory(directory = '../input/fer2013/test',
                                                  target_size = (48,48),
                                                  class_mode = 'categorical',
                                                  batch_size = 64)

In [ ]:
classes_num = 7
classes = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
Train_DIR = "../input/fer2013/train"
Test_DIR = "/content/test"

# one-hot:
def label_img(category):
    arr = np.zeros((classes_num,), dtype = int)
    arr[classes.index(category), ] = 1
    return arr

# Datasets:

img_size = 224
train = []
test = []
def create_train_set(img_size):
    for category in classes:
        imgs = 0
        category_path = os.path.join(Train_DIR, category)
        label = label_img(category)
        for img in os.listdir(category_path):
            imgs = imgs + 1
            if imgs == 4000:
                break
            img_array = cv2.resize(cv2.imread(os.path.join(category_path, img)), (img_size, img_size))
            train.append([np.array(img_array), np.array(label)])

create_train_set(img_size)
train = np.array(train)
shuffle(train)

print(train.shape)

In [ ]:
print(train[1][0])

In [ ]:
# CREATING VALIDATION SET FROM A SUBSET OF TRAINING SET:

# valTOtrain_split_ratio = 0.2  # (val / total)
# num = int(((1 - valTOtrain_split_ratio)*train.shape[0]))
# train_set = train[:num]
# val_set = train[num:]

In [ ]:
import gc
import pandas as pd
split = 21000
X_train = np.array([i[0] for i in train[:split]]).reshape(-1, img_size, img_size, 3)
y_train = np.array([i[1] for i in train[:split]])
# label = pd.DataFrame({'c0':[1,0,0,0,0,0,0], 'c1':[0,1,0,0,0,0,0], 'c2':[0,0,1,0,0,0,0], 'c3':[0,0,0,1,0,0,0], 'c1':[0,0,0,0,1,0,0], 'c2':[0,0,0,0,0,1,0], 'c3':[0,0,0,0,0,0,1]})
# label = pd.DataFrame(y_train)
# print(label)
# label = label.apply(lambda x: x.argmax(), axis=1).values
# print(label.shape)
print(X_train.shape)
print(y_train.shape)

X_val = np.array([i[0] for i in train[split:]]).reshape(-1, img_size, img_size, 3)
y_val = np.array([i[1] for i in train[split:]])

print(X_val.shape)
print(y_val.shape)

# del train
gc.collect()

In [ ]:
# print (X_val)
print(y_train)

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPool2D, Input, UpSampling2D, Flatten, BatchNormalization, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow import keras 
# resnet_model.summary()
print(len(resnet_model.layers))

In [ ]:
# Model1 using the resnet_model as the base model and fine tuning it
for layer in resnet_model.layers:
   layer.trainable = False
# for layer in resnet_model.layers[-75:]:
#    layer.trainable = True

# **IMPORTANT**
# let's visualize layer names and layer indices to see how many layers we should freeze:
# for i, layer in enumerate(resnet_model.layers):
#    print(i, layer.name)

model1 = Sequential()
model1.add(resnet_model)
model1.add(Flatten())
model1.add(BatchNormalization())
model1.add(Dropout(0.5))
model1.add(Dense(4096,activation='relu'))
model1.add(Dropout(0.5))
model1.add(Dense(2096,activation='relu'))
model1.add(Dropout(0.5))
model1.add(Dense(1094,activation='relu'))
model1.add(Dropout(0.5))

model1.add(Dense(classes_num, activation='softmax'))




# model1.add(Flatten())
# model1.add(BatchNormalization())
# model1.add(Dropout(0.5))
# model1.add(Dense(50, activation='relu'))
# model1.add(Dense(20, activation='relu'))



# model1.add(Dense(4096,activation='relu'))
# model1.add(Dropout(0.5))
# model1.add(Dense(3096,activation='relu'))
# model1.add(Dropout(0.5))
# model1.add(Dense(2096,activation='relu'))
# model1.add(Dropout(0.5))
# model1.add(Dense(1096,activation='relu'))
# model1.add(Dropout(0.5))
# model1.add(Dense(596,activation='relu'))
# model1.add(Dropout(0.5))


# model1.add(Dense(32,kernel_initializer='he_uniform'))
# model1.add(BatchNormalization())
# model1.add(Dense(32,activation='relu'))
# model1.add(Dropout(0.5))
# model1.add(Dense(32,kernel_initializer='he_uniform'))
# model1.add(BatchNormalization())
# model1.add(Dense(32,activation='relu'))
# model1.add(Dropout(0.5))
# model1.add(Dense(32,kernel_initializer='he_uniform'))
# model1.add(BatchNormalization())
# model1.add(Dense(32,activation='relu'))


# model1.add(Dense(classes_num, activation='softmax'))

model1.summary()

In [ ]:
# img_size = 224
# classes_num = 7
# from tensorflow.keras.applications.vgg16 import VGG16
# import tensorflow.keras.applications

# vgg_model = VGG16(input_shape=(img_size, img_size, 3), include_top=False, weights='imagenet')



In [ ]:
# for layer in vgg_model.layers:
#     print(layer)
#    layer.trainable = False
# for layer in resnet_model.layers[-75:]:
#    layer.trainable = True
# model1.add(vgg_model)
# model1.add(Dense(1094,activation='relu'))
# model1.add(Dropout(0.5))

# model1.add(Dense(classes_num, activation='softmax'))
# model1.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
# from keras.utils import plot_model
plot_model(model1, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
# #stacking
# #1 Random forest
# from sklearn.ensemble import RandomForestClassifier
# rf= RandomForestClassifier(n_estimators = 10,n_classes = 7)
# rf.fit(X_train, y_train,batch_size = 64, epochs=14)

# # Make predictions
# y_train_pred = rf.predict(X_train)
# y_test_pred = rf.predict(X_test)

# # Training set performance
# rf_train_accuracy = accuracy_score(y_train, y_train_pred) # Calculate Accuracy
# rf_train_mcc = matthews_corrcoef(y_train, y_train_pred) # Calculate MCC
# rf_train_f1 = f1_score(y_train, y_train_pred, average='weighted') # Calculate F1-score

# # Test set performance
# rf_test_accuracy = accuracy_score(y_test, y_test_pred) # Calculate Accuracy
# rf_test_mcc = matthews_corrcoef(y_test, y_test_pred) # Calculate MCC
# rf_test_f1 = f1_score(y_test, y_test_pred, average='weighted') # Calculate F1-score

# print('Model performance for Training set')
# print('- Accuracy: %s' % rf_train_accuracy)
# print('- MCC: %s' % rf_train_mcc)
# print('- F1 score: %s' % rf_train_f1)
# print('----------------------------------')
# print('Model performance for Test set')
# print('- Accuracy: %s' % rf_test_accuracy)
# print('- MCC: %s' % rf_test_mcc)
# print('- F1 score: %s' % rf_test_f1)


In [ ]:
model1.compile(optimizer=Adam(learning_rate=0.0002),loss='categorical_crossentropy',metrics=['accuracy'])
history1 = model1.fit(X_train, y_train, validation_split=0.1 ,batch_size = 64, epochs=50

In [ ]:
# test_dir  = "../input/fer2013/test"
# from tensorflow.keras.preprocessing.image import ImageDataGenerator 
# batch_size = 64
# target_size = (48,48)
# test_datagen  = ImageDataGenerator(rescale=1./255)
# test_generator = test_datagen.flow_from_directory(
#         test_dir,
#         target_size=target_size,
#         batch_size=batch_size,
#         color_mode="grayscale",
# #         class_mode='categorical',
#         shuffle=False)

# STEP_SIZE_TEST  = test_generator.n//test_generator.batch_size
# score = model1.evaluate(test_generator, steps=STEP_SIZE_TEST) 
# print('Test loss: ', score[0])
# print('Test accuracy: ', score[1])

In [ ]:
# models.save_model(model1, 'fer2013_resnet50_transferlearning.h5') 

In [ ]:
# import shutil
# shutil.make_archive("model_resnet_FineTuned", 'zip', "./model_resntnet_FineTuned")

In [ ]:
plt.plot(history1.history['accuracy'])
plt.plot(history1.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history1.history['loss'])
plt.plot(history1.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
classes_num = 7
classes = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
# Train_DIR = "../input/fer2013/train"
Test_DIR = "../input/fer2013/test"

# one-hot:
def label_img(category):
    arr = np.zeros((classes_num,), dtype = int)
    arr[classes.index(category), ] = 1
    return arr

# Datasets:

img_size = 224
# train = []
test = []
def create_test_set(img_size):
    for category in classes:
        imgs = 0
        category_path = os.path.join(Test_DIR, category)
        label = label_img(category)
        for img in os.listdir(category_path):
            imgs = imgs + 1
            if imgs == 4000:
                break
            img_array = cv2.resize(cv2.imread(os.path.join(category_path, img)), (img_size, img_size))
            test.append([np.array(img_array), np.array(label)])

create_test_set(img_size)
test = np.array(test)
shuffle(test)

print(test.shape)

In [ ]:
import gc
import pandas as pd
split = 21000
X_test = np.array([i[0] for i in test]).reshape(-1, img_size, img_size, 3)
y_test = np.array([i[1] for i in test])
gc.collect()

In [ ]:
# results = model1.evaluate(X_test, y_test, batch_size=64)